In [1]:
import pygame
import random
import json
import import_ipynb
import RL

pygame.init()

#Parameters

#Screen size
square_size = 21 
height_squares = 20*square_size
width_squares = 20*square_size

#game parameters
save_qvals = 10 #update qvalues every 10 games
fs = 50000 #framespeed

#Colours
chaser_colour = (255, 255, 255) #score #chaser #grid
bg_colour = (0, 0, 0) #background
target_colour = (255, 0, 0) #target

 
#loop that takes an action moves the chaser and target and updates the Q value table
def Run_game(best_score):
    global game_display
    
    game_display = pygame.display.set_mode((width_squares, height_squares))
    pygame.display.set_caption('Chaser Game')
    clock = pygame.time.Clock()

    # Chaser
    chaser_H = width_squares / 2
    chaser_V = height_squares / 2
    chaser_list = [(chaser_H, chaser_V)]
    length_of_chaser = 1
    #initialice movement variables
    H_move = 0
    V_move = 0
    

    # target
    H_target = round(random.randrange(0, width_squares - 21) / 21) * 21
    V_target = round(random.randrange(0, height_squares - 21) / 21) * 21

    
    #game loop
    done = False
    reason = None
    
    while not done:
        
      # 1) Choose action 
        action = rl.choose_action(chaser_list, (H_target,V_target))
        if action == "up":
            V_move = -21
            H_move = 0
        elif action == "left":
            H_move = -21
            V_move = 0
        elif action == "down":
            V_move = 21
            H_move = 0
        elif action == "right":
            H_move = 21
            V_move = 0
        

      # 2) Move chaser
        chaser_H += H_move
        chaser_V += V_move
        chaser_front = (chaser_H,chaser_V)
        #adds a square to the body
        chaser_list.append(chaser_front)
        
        
      # 3) Check chaser surroundings
        # check for walls
        if chaser_H >= width_squares or chaser_H < 0 or chaser_V >= height_squares or chaser_V < 0:
            reason = 'hit_wall'
            done = True

        # check for chaser body
        if chaser_front in chaser_list[:-1]:
            reason = 'hit_body'
            done = True

        # check if chaser found target
        if chaser_H == H_target and chaser_V == V_target:
            #change target position to a new random position
            H_target = round(random.randrange(0, width_squares - square_size) / 21) * 21
            V_target = round(random.randrange(0, height_squares - square_size) / 21) * 21
            #add to the chaser body when the chaser finds the target
            length_of_chaser += 1

        
      # 4) Draw movements & changes on the screen
        
        #removes a square from the body unless target was reached (we make the chaser body longer when it meets a target to increase difficulty)
        if len(chaser_list) > length_of_chaser: 
            del chaser_list[0]

        # update chaser, target and score on display
        #display
        game_display.fill(bg_colour)
        #draw grid
        draw_grid(height_squares, width_squares, square_size)
        #draw chaser
        draw_chaser(chaser_list)
        # draw target
        draw_target(H_target, V_target)
        #write current score and best score
        score=length_of_chaser - 1
        draw_score(score, best_score)
        pygame.display.update()

      # 5) Update Q values table
        rl.New_q_vals(reason)
        
      # Set up the next frame
        clock.tick(fs)

    return score, reason

def draw_grid(height_squares, width_squares, square_size):
    for h in range(0, width_squares, square_size):
        for v in range(0, height_squares, square_size):
            rect = pygame.Rect(h, v, square_size, square_size)
            pygame.draw.rect(game_display, chaser_colour, rect, 1)

def draw_chaser(chaser_list):
    for body_part in chaser_list:
        pygame.draw.rect(game_display, chaser_colour, [body_part[0], body_part[1], square_size, square_size])

def draw_target(H_target, V_target):
    pygame.draw.rect(game_display, target_colour, [H_target, V_target, square_size, square_size])   

def draw_score(score, best_score):
    font = pygame.font.SysFont("comicsansms", 35)
    value = font.render(f"Score: {score}   Best: {best_score}", True, chaser_colour)
    game_display.blit(value, [0, 0])





#RUN GAME
cnt = 1 #game count

rl = RL.RL(height_squares, width_squares, square_size)
best_score=0
results=[]

while True:
    rl.Reset()
    if cnt > 100: #after 100 games we only use explotation
        rl.e = 0
    else:
        rl.e = 0.1
    score, reason = Run_game(best_score)
    #set best score
    if score>=best_score: 
        best_score=score
    #record results
    results.append({'game':cnt, 'score':score })
    #add to game count 
    cnt += 1
    if cnt % save_qvals == 0: # Save qvalues every qvalue_dump_n games
        #print("Save Qvals")
        rl.write_qvalues()
        

pygame 2.0.1 (SDL 2.0.14, Python 3.7.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
importing Jupyter notebook from RL.ipynb


KeyboardInterrupt: 

In [ ]:
#save game run results
with open("results50.json", "w") as f:
    json.dump(results, f)